[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/Sistemas-Multimedia/Sistemas-Multimedia.github.io/HEAD?labpath=contents%2FPNG%2Fdisplay_video.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/contents/PNG/display_video.ipynb)

# Display a sequence of video

In [ ]:
%%bash
if [ -d "$HOME/repos" ]; then
    echo "\"$HOME/repos\" exists"
else
    mkdir ~/repos
    echo Created $HOME/repos
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/MRVC" ]; then
    cd $HOME/repos/MRVC
    echo "$HOME/repos/MRVC ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/Sistemas-Multimedia/MRVC.git
fi

In [ ]:
#%%bash
#if [ -d "$HOME/repos/image_IO" ]; then
#    cd $HOME/repos/image_IO
#    echo "$HOME/repos/image_IO ... "
#    git pull 
#else
#    cd $HOME/repos
#    git clone https://github.com/vicente-gonzalez-ruiz/image_IO.git
#fi
try:
    import image_IO
except:
    !pip install 'image_IO @ git+https://github.com/vicente-gonzalez-ruiz/image_IO'

In [ ]:
import os

try:
    import colored
except:
    !pip install colored
    import colored

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

try:
    import cv2
except:
    !pip install opencv-python
    !pip install opencv-python-headless # Binder compatibility
    import cv2

try:
    import matplotlib.pyplot as plt
except:
    !pip install matplotlib
    import matplotlib.pyplot as plt
%matplotlib inline

!ln -sf ~/repos/image_IO/image_3.py .
!ln -sf ~/repos/image_IO/logging_config.py .

#import image_3 as frame
from image_IO import image_3 as frame

In [ ]:
if 'google.colab' in str(get_ipython()):
    !apt-get -qq install -y bc ffmpeg
# Binder install the pakages indicated in "apt.txt" placed at the root directory of the repo.

In [ ]:
def show(img, title):
    img = frame.normalize(img)
    plt.figure(figsize=(10,10))
    plt.title(title, fontsize=20)
    plt.imshow(img)

def show_seq(prefix="/tmp/", first_frame = 0, N_frames=5):
    for i in range(first_frame, first_frame + N_frames):
        img = frame.read(prefix, i)
        show(img, f"{prefix}{i:03d}")

def show_motion(prefix="/tmp/", first_frame = 0, N_frames=5):
    plt.figure(figsize=(10,10))
    prev = frame.read(prefix, first_frame)
    plt.title(f"{prefix}{first_frame:03d}", fontsize=20)
    plt.imshow(frame.normalize(prev))
    for i in range(first_frame + 1, first_frame + N_frames, 1):
        img = frame.read(prefix, i)
        diff = img.astype(np.int16) - prev
        prev = img
        diff = frame.normalize(diff)
        plt.figure(figsize=(10,10))
        plt.title(f"{prefix}{i:03d}", fontsize=20)
        plt.imshow(diff)
        #plt.imshow(diff[0])

In [ ]:
home = os.environ["HOME"]

In [ ]:
!rm -f /tmp/*.png

In [ ]:
sequence = "stockholm_1280x768x50x420x578.avi"
URL = "http://www.hpca.ual.es/~vruiz/videos/"
N_frames = 100 # Min val 3
first_frame = 2

In [ ]:
prefix="/tmp/original_"
!$home/repos/MRVC/sequences/extract_frames.sh -u $URL -s $sequence -o $prefix -f $first_frame -n $N_frames

In [ ]:
#home = os.environ["HOME"]
#sequence_path = home + "/repos/MRVC/sequences/stockholm/"

In [ ]:
img = frame.read(prefix, 2)
show(img, "stockholm002")

In [ ]:
show_seq(prefix, first_frame = first_frame)

In [ ]:
show_motion(prefix, first_frame = first_frame)

## Correlation between the number of bits/pixel and the entropy

In [ ]:
%%bash
if [ -d "$HOME/repos/information_theory" ]; then
    cd $HOME/repos/information_theory
    echo "$HOME/information_theory ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/information_theory.git
fi

In [ ]:
!ln -sf ~/repos/information_theory/information.py .
import information
import os

In [ ]:
def compute_BPPs_of_sequence(prefix="/tmp/", first_frame = 0, N_frames=5):
    BPPs = []
    for i in range(first_frame, first_frame + N_frames):
        #img = frame.read(prefix, i)
        number_of_bytes = os.path.getsize(f"{prefix}{i:03d}.png")
        number_of_pixels = img.shape[0]*img.shape[1]
        BPP = (number_of_bytes*8) / number_of_pixels
        BPPs.append(BPP)
    return BPPs

BPPs = compute_BPPs_of_sequence(prefix, first_frame, N_frames)
BPPs

In [ ]:
def compute_entropies_of_sequence(prefix="/tmp/", first_frame = 0, N_frames=5):
    entropies = []
    for i in range(first_frame, first_frame + N_frames):
        img = frame.read(prefix, i)
        entropy = information.entropy(img.flatten())*img.shape[2]
        entropies.append(entropy)
    return entropies

entropies = compute_entropies_of_sequence(prefix, first_frame, N_frames)
entropies

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(BPPs, entropies)
plt.xlabel('PNG (bits/pixel)')
plt.ylabel('entropy (bits/pixel)')
plt.show()

## Conclusions

1. The PNG's compression ratio (measured in bits/pixel) and the entropy (computed independently for each color channel) can be considered [linearly](https://en.wikipedia.org/wiki/Linear_relation) [correlated](https://en.wikipedia.org/wiki/Correlation).

2. When the previous conclusion occurs, it is also true that if we divide the images into two different parts, the PNG's CR and the [information](https://en.wikipedia.org/wiki/Entropy_(information_theory)#Introduction) of such pars are linearly correlated. Therefore, continuing with this reasoning (splitting the data into smaller parts), we get that the contribution of a pixel to the bit-rate using PNG should be proportional to the information of the pixel.